In [1]:
from lib import geoSearch
import json

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
from shapely.geometry import mapping, shape

In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"search_calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [8]:
text = "chana guana paullier"
print(analyzedText(text))

chana guana paullier


In [ ]:
matches = GS.complete_search(text,500)
for nombre, match in matches[:4]:
    if match and len(match['objects']) == 1:
        shape_1 = shape(match['objects'][0]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
        #print(json.dumps(mapping(shape_1.representative_point())))
    elif match and len(match['objects']) > 1:
        shape_1 = shape(match['objects'][0]['geometry'])
        shape_2 = shape(match['objects'][1]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
        #print mapping(shape_1.intersection(shape_2).representative_point())
print('\n')

In [14]:
def boosting_match_bool_search2(text, size=1000, boost=2, negative_boost=0.5):
    """
    Elasticsearch query. I'm trying to boost results that doesn't come from negative_types or must_not terms.
    """
    return es.search(index='montevideo', body=
    {
        "from": 0, "size": size,
        "query": {

                    "bool": {
                        "should": [
                            { "match": { "nombre":            text }},
                            { "match": { "nombre.variant_1":  text }},
                            { "match": { "nombre.variant_2":  text }},
                            { "match": { "nombre_1":            text }},
                            { "match": { "nombre_1.variant_1":  text }},
                            { "match": { "nombre_1.variant_2":  text }},
                            { "match": { "aliases":           text }},
                            { "match": { "aliases.variant_1": text }},
                            { "match": { "aliases.variant_2": text }}
                          ]
                    }
                },
            
        
         "highlight": {
                "fields": {
                    "*": {}
                },
             "number_of_fragments": 10
            }
    })['hits']['hits']

In [15]:
def get_result_object(result):
    result_geo_type = result['_source']['geometry']['type']
    result_id = result['_id']
    result_score = result['_score']
    result_geometry = result['_source']['geometry']
    result_name = 'NO_NAME'
    result_name_1 = None
    if result['_source'].get('nombre', None):
        result_name = result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
    elif result['_source'].get('aliases', None):
        result_name = result['_source']['aliases'].encode('ascii', 'ignore').decode('ascii')
    if result['_source'].get('nombre_1', None):
        result_name_1 = result['_source']['nombre_1'].encode('ascii', 'ignore').decode('ascii')
    result_striped_name = GS.strip_name(result_name)
    result_striped_name_1 = GS.strip_name(result_name_1)
    result_highlights = []
    highlights = result.get('highlight', None)
    if highlights:
        for hg_values in highlights.values():
            result_highlights = result_highlights + re.findall("<em>(.*?)<\/em>", ' '.join(hg_values))
    result_highlights = list(set(result_highlights))
    return {
        'id': result_id, 
        'geo_type': result_geo_type, 
        'name': result_name, 
        's_name': result_striped_name,
        'name_1': result_name_1, 
        's_name_1': result_striped_name_1,
        'score': result_score, 
        'geometry': result_geometry, 
        'highlights': result_highlights}

In [16]:
import re
for res in boosting_match_bool_search2(text):
    obj = get_result_object(res)
    print(obj['name'],obj['name_1'],obj['score'],obj['highlights'],obj['geo_type'])

(u'GUANA', u'JUAN PAULLIER', 38.409744, [u'GUANA', u'PAULLIER'], u'Point')
(u'JUAN PAULLIER', u'GUANA', 38.188137, [u'GUANA', u'PAULLIER'], u'Point')
(u'CHANA', u'JUAN PAULLIER', 37.068462, [u'CHANA', u'PAULLIER'], u'Point')
(u'JUAN PAULLIER', u'CHANA', 36.813293, [u'CHANA', u'PAULLIER'], u'Point')
(u'GUANA', u'BV GRAL ARTIGAS', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'BV GRAL ARTIGAS', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'DR MARIO CASSINONI', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'ACEVEDO DIAZ', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'DR JOAQUIN REQUENA', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'JUAN MANUEL BLANES', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'DR PABLO DE MARIA', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'JUAN D JACKSON', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'EDUARDO ACEVEDO', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', u'DR JOAQUIN DE SALTERAIN', 31.32797, [u'GUANA'], u'Point')
(u'GUANA', None, 31.32797, [u'GUANA'], u'MultiLineStrin